# CNN-to-pridect-Liver-Disease


By Aziz Alreshidi
Email : swe2010-m-s@hotmail.com


Abstract
Explore and compare machine learning techniques and tools used to predict liver disease for patients in India utilizing a set of health data measurements.
Description of data source and web links
The Indian Liver Patient Dataset (ILPD) contains 10 health variables for Indian patients along with a binary outcome variable indicating whether or not the patient has a liver disease.  The data set was published on 2012-05-21 by two professors and an associate computer science research professor.  

University of California, Irvine:  Center for Machine Learning and Intelligent Systems
ILPD (Indian Liver Patient Dataset) Data Set
https://archive.ics.uci.edu/ml/datasets/ILPD+(Indian+Liver+Patient+Dataset)#

Kaggle: UCI Machine Learning
Indian Liver Patient Records
https://www.kaggle.com/uciml/indian-liver-patient-records/version/1


Number of records and size of the dataset
This dataset contains a total of 583 records.  416 records are for patients that have a liver disease apart from 167 patients who do not have a liver disease.  The dataset is presented in CSV format with a file size of 24kb.	

Number and description of attributes in the dataset
10 features are recorded for these patients as well as classification labels of disease/no disease.


https://imgur.com/a/hnlsm0P

# Importing Libraries

In [ ]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import scikitplot as skplt
from keras.layers.convolutional import Convolution2D
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff 
import  plotly.offline as py
import plotly.graph_objs as go 

from plotly.offline import download_plotlyjs,init_notebook_mode, iplot, plot
from plotly import tools 
py.init_notebook_mode(connected = True)

import cufflinks as cf 
cf.go_offline()


# Importing Data


In [ ]:
df = pd.read_csv("../input/indian_liver_patient.csv")



In [ ]:
df.info()

In [ ]:
df.plot(kind='kde')



# Datatypes and its distribution

In [ ]:
Data_types = df.dtypes.value_counts()
print(Data_types)

plt.figure(figsize = (14,4))
sns.barplot(x = Data_types.index, y = Data_types.values)
plt.title("Data Type Distribution")

# Checking NaN¶ value  

In [ ]:
df.columns[df.isnull().any()].tolist()




# We Have Null Value in  Albumin_and_Globulin_Ratio Fill in with mean 0.6

In [ ]:
df["Albumin_and_Globulin_Ratio"].fillna("0.6", inplace = True) 



# Checking Null 

In [ ]:
df.isnull().sum()




# # Target distribution¶
# Number of patients diagnosed with liver disease:  416 is 71.4%
# Number of patients  diagnosed with no liver disease:  167 is 28.6%


In [ ]:
Liver = df[(df['Dataset'] == 1)]
NoLiver = df[(df['Dataset'] != 1)]

trace = go.Pie(labels = ['Liver Disease', 'No Liver Disease'], values = df['Dataset'].value_counts(), 
               textfont=dict(size=15), opacity = 0.8,
               marker=dict(colors=['lightskyblue','gold'], 
                           line=dict(color='#000000', width=1.5)))


layout = dict(title =  'Distribution of Number  patients')
           
fig = dict(data = [trace], layout=layout)
py.iplot(fig)

# Gender Distrbution 
# There are patients who are male:  441
#  There are patients who are female:  142
# unbalnce

In [ ]:
Male = df[(df['Gender'] == 1)]
Female = df[(df['Gender'] != 1)]

trace = go.Pie(labels = ['Male', 'Female'], values = df['Gender'].value_counts(), 
               textfont=dict(size=15), opacity = 0.8,
               marker=dict(colors=['lightskyblue','gold'], 
                           line=dict(color='#000000', width=1.5)))


layout = dict(title =  'Distribution of Gender patients')
           
fig = dict(data = [trace], layout=layout)
py.iplot(fig)

# Corrolation  Features

In [ ]:
liver_corr = df.corr()

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(liver_corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'YlGnBu')
plt.title('Correlation between features');

In [ ]:

corr=df.select_dtypes(include=[np.number]).dropna().corr()
mask=np.zeros_like(corr)
mask[np.triu_indices_from(mask)]=True
sns.heatmap(corr,cmap=sns.diverging_palette(256,0,sep=80,n=7,as_cmap=True), annot=True, mask=mask)

# Visualize the correlation of the numerical variables

In [ ]:
sns.pairplot(df.select_dtypes(include=[np.number]), dropna=True)



In [ ]:
#correlation
correlation = df.corr()
#tick labels
matrix_cols = correlation.columns.tolist()
#convert to array
corr_array  = np.array(correlation)

In [ ]:
#Plotting
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   xgap = 2,
                   ygap = 2,
                   colorscale='Viridis',
                   colorbar   = dict() ,
                  )
layout = go.Layout(dict(title = 'Correlation Matrix for variables',
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                     ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9)),
                       )
                  )
fig = go.Figure(data = [trace],layout = layout)
py.iplot(fig)

# This Plot to visualize how many do we have patients Male and Female with liver and no Liver Disease

In [ ]:

Liver = df[(df['Dataset'] == 1)]
NoLiver = df[(df['Dataset'] != 1)]



In [ ]:
#------------COUNT-----------------------
trace = go.Bar(x = (len(Liver), len(NoLiver)), y = ['with liver disease', 'NO liver disease'], orientation = 'h', opacity = 0.8, marker=dict(
        color=[ 'gold', 'lightskyblue'],
        line=dict(color='#000000',width=1.5)))

layout = dict(title =  'Count of patients Gender variable')
                    
fig = dict(data = [trace], layout=layout)
py.iplot(fig)

#------------PERCENTAGE-------------------
trace = go.Pie(labels = ['NO liver disease', 'with liver disease'], values = df['Dataset'].value_counts(), 
               textfont=dict(size=15), opacity = 0.8,
               marker=dict(colors=['lightskyblue', 'gold'], 
                           line=dict(color='#000000', width=1.5)))


layout = dict(title =  'Distribution of diagnosis variable')
           
fig = dict(data = [trace], layout=layout)
py.iplot(fig)

In [ ]:
def plot_feat1_feat2(feat1, feat2) :  
    trace0 = go.Scatter(
        x = Liver[feat1],
        y = Liver[feat2],
        name = 'Yes',
        mode = 'markers', 
        marker = dict(color = '#FFD700',
            line = dict(
                width = 1)))

    trace1 = go.Scatter(
        x = NoLiver[feat1],
        y = NoLiver[feat2],
        name = 'NO',
        mode = 'markers',
        marker = dict(color = '#7EC0EE',
            line = dict(
                width = 1)))

    layout = dict(title = feat1 +" "+"vs"+" "+ feat2,
                  yaxis = dict(title = feat2,zeroline = False),
                  xaxis = dict(title = feat1, zeroline = False)
                 )

    plots = [trace0, trace1]

    fig = dict(data = plots, layout=layout)
    py.iplot(fig)

# Notice the direct relationship between Total_Bilirubin and Direct_Bilirubin.

In [ ]:
plot_feat1_feat2('Total_Bilirubin','Direct_Bilirubin')
plot_feat1_feat2('Direct_Bilirubin','Total_Bilirubin')
plot_feat1_feat2('Alkaline_Phosphotase','Alamine_Aminotransferase')
plot_feat1_feat2('Albumin','Total_Protiens')
plot_feat1_feat2('Albumin','Age')










# No relationship between Alkaline_Phosphotase and Alamine_Aminotransferase

In [ ]:
plot_feat1_feat2('Alkaline_Phosphotase','Alamine_Aminotransferase')
plot_feat1_feat2('Age','Total_Protiens')
plot_feat1_feat2('Albumin_and_Globulin_Ratio','Alkaline_Phosphotase')
plot_feat1_feat2('Albumin','Direct_Bilirubin')


# Data Set Information:
This data set contains 416 liver patient records and 167 non liver patient records.The data set was collected from north east of Andhra Pradesh, India. Selector is a class label used to divide into groups(liver patient or not). This data set contains 441 male patient records and 142 female patient records.
Any patient whose age exceeded 89 is listed as being of age "90".
Attribute Information:
Age Age of the patient
Gender Gender of the patient
TB Total Bilirubin
DB Direct Bilirubin
Alkphos Alkaline Phosphotase
Sgpt Alamine Aminotransferase
Sgot Aspartate Aminotransferase
TP Total Protiens
ALB Albumin
A/G Ratio Albumin and Globulin Ratio
Selector field used to split the data into two sets (labeled by the experts)

In [ ]:
df_sex = pd.get_dummies(df['Gender'])
df_new = pd.concat([df, df_sex], axis=1)
Droop_gender = df_new.drop(labels=['Gender' ],axis=1 )
Droop_gender.columns = ['Age', 'Total_Bilirubin', 'Direct_Bilirubin','Alkaline_Phosphotase','Alamine_Aminotransferase','Aspartate_Aminotransferase','Total_Protiens','Albumin','Albumin_and_Globulin_Ratio','Male','Fmale','Dataset']

X = Droop_gender.drop('Dataset',axis=1)
y = Droop_gender['Dataset']


# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)



# StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# CNN Keras Sequantial

# Using Convolutional Neural Networks (CNN)¶
# Here is the CNN i put the input_diminetion as 11 becouse we have 11 features.

In [ ]:
classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# compile ANN
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the data
hisroy =classifier.fit(X_train, y_train, batch_size = 20, epochs = 50)

# Plotting Model Performance 

In [ ]:

plt.plot(hisroy.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hisroy.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.style.use("ggplot")
plt.figure()
N = 50
plt.plot(np.arange(0, N), hisroy.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), hisroy.history["acc"], label="train_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

In [ ]:

y_pred = classifier.predict(X_test)
y_pred = [ 1 if y>=0.5 else 0 for y in y_pred ]




# Print Classification Report

In [ ]:
print(classification_report(y_test, y_pred))




# plot_confusion_matrix

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True)




# you found this notebook helpful or you just liked it , some upvotes would be very much appreciated - That will keep me motivated. Please let me know if you have quetions